In [1]:
import pandas as pd 
import numpy as np

In [3]:
def trim_data(df):
    
    df['epitope'] = df['epitope'].apply(lambda x: x[2:7])
    df['CDR3B'] = df['CDR3B'].apply(lambda x: x[4:14])
    
    return df

In [ ]:
DATA_PATH = '/home/jhenderson/Documents/Projects/data_sets/tcr_sequences/CI_CRUK_datathon/'
#DATA_PATH = '/Users/james/Documents/Projects/data_sets/tcr_sequences/CI_CRUK_datathon/'

train = trim_data(pd.read_csv(DATA_PATH + 'processed_data/train_fixed_lengths.csv'))
